In [1]:
import netrc
info = netrc.netrc()

In [2]:
from qat.lang.AQASM import Program, H, X, CNOT, RX, I, RY, RZ, CSIGN #Gates
from qat.core import Observable, Term, Batch #Hamiltonian
import numpy as np
from qat.plugins import ScipyMinimizePlugin
from qat.hardware import make_depolarizing_hardware_model
import pickle

c:\Users\cqtv201\AppData\Local\anaconda3\envs\atos\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
nqbt = 5 # number of qubits
nruns = 0 #nbshots for observable sampling

In [4]:
base = 1
expo = 5

In [5]:
dep = np.arange(1, 11, 1, dtype = int)

In [6]:
#Instantiation of Hamiltoniian
heisen = Observable(nqbt)
#Generation of Heisenberg Hamiltonian
for q_reg in range(nqbt-1):
    heisen += Observable(nqbt, pauli_terms = [Term(1., typ, [q_reg,q_reg + 1]) for typ in ['XX','YY','ZZ']])

In [7]:
#exact calculation for ground state
from qat.fermion import SpinHamiltonian

heisen_class = SpinHamiltonian(nqbits=heisen.nbqbits, terms=heisen.terms)
heisen_mat = heisen_class.get_matrix()

eigvals, eigvecs = np.linalg.eigh(heisen_mat)
g_energy = eigvals[0]
g_state = eigvecs[:,0]

In [8]:
#from qlmaas.plugins import SeqOptim
#optimizer_scipy = SeqOptim()
#from qlmaas.plugins import ScipyMinimizePlugin
#optimizer_scipy = ScipyMinimizePlugin(method="COBYLA", tol=1e-4, options={"maxiter": 10000})
from qat.vsolve.optimize import ScipyMinimizePlugin
optimizer_scipy = ScipyMinimizePlugin(method="COBYLA",
                                  tol=1e-6,
                                  options={"maxiter": 25000},
#                                  x0=np.zeros(nqbt)
                                     )

In [9]:
#infidelity
F = base*(10**(-expo))
custom_hardware = make_depolarizing_hardware_model(eps1=F, eps2=2*F, correl_type='single_qubit')


In [11]:
#from qlmaas.qpus import LinAlg
#from qat.qpus import get_default_qpu
from qlmaas.qpus import LinAlg
qpu_noisy = NoisyQProc(hardware_model = custom_hardware, fidelity=0.999)

ImportError: Could not connect to the QLMaaS server. Please ensure a QLMaaS configuration file exists on your computer

In [ ]:
base = 1
expo = 5

In [ ]:
#variational gates
E_val = []
for ct in dep:
    qprog = Program()
    qbits = qprog.qalloc(nqbt)
    #variational parameters used for generating gates (permutation of [odd/even, xx/yy/zz])
    ao = [qprog.new_var(float, 'ao_%s'%i) for i in range(ct)]
    bo = [qprog.new_var(float, 'bo_%s'%i) for i in range(ct)]
    co = [qprog.new_var(float, 'co_%s'%i) for i in range(ct)]
    ae = [qprog.new_var(float, 'ae_%s'%i) for i in range(ct)]
    be = [qprog.new_var(float, 'be_%s'%i) for i in range(ct)]
    ce = [qprog.new_var(float, 'ce_%s'%i) for i in range(ct)]
    for q_index in range(nqbt):
        X(qbits[q_index])
    for q_index in range(nqbt):
        if not q_index%2 and q_index <= nqbt-1:
            H(qbits[q_index])
    for q_index in range(nqbt):
        if not q_index%2 and q_index <= nqbt-2:
            CNOT(qbits[q_index],qbits[q_index+1])
    for it in range(ct):
        for q_index in range(nqbt): #odd Rzz
            if q_index%2 and q_index <= nqbt-2:
                CNOT(qbits[q_index],qbits[q_index+1])
                RZ(ao[it-1]/2)(qbits[q_index+1])
                #I(qbits[q_index])
                CNOT(qbits[q_index],qbits[q_index+1])
        for q_index in range(nqbt): #odd Ryy
            if q_index%2 and q_index <= nqbt-2:
                RZ(np.pi/2)(qbits[q_index])
                #I(qbits[q_index])
                RZ(np.pi/2)(qbits[q_index+1])
                #I(qbits[q_index])
                H(qbits[q_index])
                H(qbits[q_index+1])
                CNOT(qbits[q_index],qbits[q_index+1])
                RZ(bo[it-1]/2)(qbits[q_index+1])
                #I(qbits[q_index])
                CNOT(qbits[q_index],qbits[q_index+1])
                H(qbits[q_index])
                H(qbits[q_index+1])
                RZ(-np.pi/2)(qbits[q_index])
                #I(qbits[q_index])
                RZ(-np.pi/2)(qbits[q_index+1])
                #I(qbits[q_index])
        for q_index in range(nqbt): #odd Rxx
            if q_index%2 and q_index <= nqbt-2:
                H(qbits[q_index])
                H(qbits[q_index+1])
                CNOT(qbits[q_index],qbits[q_index+1])
                RZ(co[it-1]/2)(qbits[q_index+1])
                #I(qbits[q_index])
                CNOT(qbits[q_index],qbits[q_index+1])
                H(qbits[q_index])
                H(qbits[q_index+1])
        for q_index in range(nqbt): #even Rzz
            if not q_index%2 and q_index <= nqbt-2:
                CNOT(qbits[q_index],qbits[q_index+1])
                RZ(ae[it-1]/2)(qbits[q_index+1])
                #I(qbits[q_index])
                CNOT(qbits[q_index],qbits[q_index+1])
        for q_index in range(nqbt): #even Ryy
            if not q_index%2 and q_index <= nqbt-2:
                RZ(np.pi/2)(qbits[q_index])
                #I(qbits[q_index])
                RZ(np.pi/2)(qbits[q_index+1])
                #I(qbits[q_index])
                H(qbits[q_index])
                H(qbits[q_index+1])
                CNOT(qbits[q_index],qbits[q_index+1])
                RZ(be[it-1]/2)(qbits[q_index+1])
                #I(qbits[q_index])
                CNOT(qbits[q_index],qbits[q_index+1])
                H(qbits[q_index])
                H(qbits[q_index+1])
                RZ(-np.pi/2)(qbits[q_index])
                #I(qbits[q_index])
                RZ(-np.pi/2)(qbits[q_index+1])
                #I(qbits[q_index])
        for q_index in range(nqbt): #even Rxx
            if not q_index%2 and q_index <= nqbt-2:
                H(qbits[q_index])
                H(qbits[q_index+1])
                CNOT(qbits[q_index],qbits[q_index+1])
                RZ(ce[it-1]/2)(qbits[q_index+1])
                #I(qbits[q_index])
                CNOT(qbits[q_index],qbits[q_index+1])
                H(qbits[q_index])
                H(qbits[q_index+1])
    circuit = qprog.to_circ()
    #Ng1, Ng2 = gate_ct(circuit)
    #Ng = Ng1 + Ng2
    custom_hardware = make_depolarizing_hardware_model(eps1=F, eps2=2*F, correl_type='single_qubit')
    stack_noisy = optimizer_scipy | qpu_noisy
    #job at hand, not the observable class but instantiated job
    job = circuit.to_job(observable = heisen, nbshots = 0)
    res_ideal = stack_noisy.submit(job)
    #res_ide = res_ideal.join()
    E_val.append(res_ideal)

In [ ]:
#with open('global_noisy/noisy-%s_%s.pkl'%(base, expo), 'wb') as file:
with open('LOCAL_noisy-%s_%s.pkl'%(base, expo), 'wb') as file:
    pickle.dump(E_val, file)